# Importando Bibliotecas

In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00


In [ ]:
import requests
import os
import pandas as pd
from IPython.display import IFrame
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import re
import optuna
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Preparando os arquivos

In [ ]:
# URL base da API GitHub
url_base = "https://api.github.com/repos/PedroCosDi/estudos_ia/contents/aprendizado_supervisionado/queimadas_serie_temporal/dados/processados"

# Cabeçalho opcional (caso precise de autenticação para evitar limites)
headers = {"Accept": "application/vnd.github.v3+json"}

# Pasta de destino no Colab
pasta_destino = "/content/processados/"
os.makedirs(pasta_destino, exist_ok=True)

def listar_arquivos_git(url):
    """Lista os arquivos na pasta do repositório."""
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao acessar a API do GitHub: {response.status_code}")
        return []

def baixar_arquivo(url, caminho):
    """Baixa um arquivo a partir de uma URL."""
    response = requests.get(url)
    if response.status_code == 200:
        with open(caminho, 'wb') as file:
            file.write(response.content)
        print(f"Arquivo baixado: {caminho}")
    else:
        print(f"Erro ao baixar {url}. Status: {response.status_code}")

# Listar arquivos
arquivos = listar_arquivos_git(url_base)

# Baixar cada arquivo
for arquivo in arquivos:
    if arquivo["type"] == "file":
        url_download = arquivo["download_url"]
        nome_arquivo = arquivo["name"]
        caminho_arquivo = os.path.join(pasta_destino, nome_arquivo)
        baixar_arquivo(url_download, caminho_arquivo)

Arquivo baixado: /content/processados/AC_final.csv
Arquivo baixado: /content/processados/AL_final.csv
Arquivo baixado: /content/processados/AM_final.csv
Arquivo baixado: /content/processados/AP_final.csv
Arquivo baixado: /content/processados/BA_final.csv
Arquivo baixado: /content/processados/CE_final.csv
Arquivo baixado: /content/processados/DF_final.csv
Arquivo baixado: /content/processados/ES_final.csv
Arquivo baixado: /content/processados/GO_final.csv
Arquivo baixado: /content/processados/MA_final.csv
Arquivo baixado: /content/processados/MG_final.csv
Arquivo baixado: /content/processados/MS_final.csv
Arquivo baixado: /content/processados/MT_final.csv
Arquivo baixado: /content/processados/PA_final.csv
Arquivo baixado: /content/processados/PB_final.csv
Arquivo baixado: /content/processados/PE_final.csv
Arquivo baixado: /content/processados/PI_final.csv
Arquivo baixado: /content/processados/PR_final.csv
Arquivo baixado: /content/processados/RJ_final.csv
Arquivo baixado: /content/proce

In [ ]:
# Caminho da pasta onde os CSVs foram baixados
pasta_csv = '/content/processados'

# Criar dicionário de DataFrames com os mesmos nomes dos arquivos CSV
dataframes = {
    os.path.splitext(arquivo)[0]: pd.read_csv(os.path.join(pasta_csv, arquivo), index_col=0, parse_dates=[0])
    for arquivo in os.listdir(pasta_csv) if arquivo.endswith('.csv')
}

# Obter o primeiro DataFrame do dicionário
df = list(dataframes.values())[0]

# Exibir o head e as informações do primeiro DataFrame
print("Head do primeiro DataFrame:")
print(df.head())
print("\nInfo do primeiro DataFrame:")
print(df.info())

Head do primeiro DataFrame:
            PRECIPITAÇÃO TOTAL, MENSAL (MM)  \
2019-01-31                         0.205645   
2019-02-28                         0.555357   
2019-03-31                         0.616935   
2019-04-30                         0.433472   
2019-05-31                         0.228091   

            PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, MENSAL (MB)  \
2019-01-31                                        1010.653360      
2019-02-28                                        1010.136310      
2019-03-31                                        1010.774866      
2019-04-30                                        1010.376875      
2019-05-31                                        1009.972782      

            PRESSÃO ATMOSFERICA MAX.MENSAL (AUT) (MB)  \
2019-01-31                                1010.942608   
2019-02-28                                1010.436607   
2019-03-31                                1011.083737   
2019-04-30                                1010.68166

In [ ]:
print(list(dataframes.keys()))

['PA_final', 'MS_final', 'TO_final', 'AM_final', 'RR_final', 'PR_final', 'RN_final', 'AC_final', 'AL_final', 'PB_final', 'SE_final', 'CE_final', 'DF_final', 'SP_final', 'MA_final', 'PI_final', 'RS_final', 'AP_final', 'SC_final', 'GO_final', 'RJ_final', 'RO_final', 'PE_final', 'MG_final', 'MT_final', 'ES_final', 'BA_final']


In [ ]:
# Lista de colunas a serem selecionadas
selected_columns = [
    "PRESSÃO ATMOSFERICA MAX.MENSAL (AUT) (MB)",
    "TEMPERATURA ORVALHO MIN. MENSAL (AUT) (°C)",
    "UMIDADE REL. MIN. MENSAL (AUT) (%)",
    "VENTO, VELOCIDADE MENSAL (M/S)",
    "RADIACAO GLOBAL (KJ/M²)",
    "PRECIPITAÇÃO TOTAL, MENSAL (MM)",
    "TEMPERATURA MÍNIMA MENSAL (AUT) (°C)",
    "FOCOS DE QUEIMADA"
]

# Passando todos os DataFrames pela transformação
for nome_df, df in dataframes.items():
    try:
        # Selecionando as colunas especificadas
        dataframes[nome_df] = df[selected_columns]
        # Certificando-se que o índice é datetime
        dataframes[nome_df].index = pd.to_datetime(dataframes[nome_df].index)
    except KeyError as e:
        print(f"Erro em {nome_df}: {e}")

# Verificando se a transformação foi aplicada corretamente
print("DataFrames após a transformação:")
for nome_df, df in dataframes.items():
    print(f"\n{nome_df}:")
    print(df.head())  # Exibindo o head de cada DataFrame transformado

DataFrames após a transformação:

PA_final:
            PRESSÃO ATMOSFERICA MAX.MENSAL (AUT) (MB)  \
2019-01-31                                1010.942608   
2019-02-28                                1010.436607   
2019-03-31                                1011.083737   
2019-04-30                                1010.681667   
2019-05-31                                1010.265524   

            TEMPERATURA ORVALHO MIN. MENSAL (AUT) (°C)  \
2019-01-31                                   22.748387   
2019-02-28                                   22.705208   
2019-03-31                                   22.641801   
2019-04-30                                   22.988889   
2019-05-31                                   23.151277   

            UMIDADE REL. MIN. MENSAL (AUT) (%)  \
2019-01-31                           79.350806   
2019-02-28                           82.431548   
2019-03-31                           83.844086   
2019-04-30                           83.320833   
2019-05-31    

In [ ]:
# Aplicando a transformação para todos os DataFrames na lista
def process_dataframes(dataframes):
    for nome_df, df in dataframes.items():
        try:
            # Certificar-se de que o índice é datetime
            if not isinstance(df.index, pd.DatetimeIndex):
                df.index = pd.to_datetime(df.index)

            # Extrair mês e ano do índice
            df['Mes'] = df.index.month
            df['Ano'] = df.index.year

            # Codificação cíclica do mês
            df['Mes_rad'] = 2 * np.pi * df['Mes'] / 12
            df['Mes_sin'] = np.sin(df['Mes_rad'])
            df['Mes_cos'] = np.cos(df['Mes_rad'])

            # Codificar ano como diferença (ex.: 2019 = 0, 2020 = 1, ...)
            df['Ano_diff'] = df['Ano'] - df['Ano'].min()

            # Remover colunas auxiliares
            df.drop(columns=['Mes', 'Mes_rad', 'Ano'], inplace=True)
        except Exception as e:
            print(f"Erro ao processar {nome_df}: {e}")
    return dataframes

# Chamando a função
process_dataframes(dataframes)

{'PA_final':             PRESSÃO ATMOSFERICA MAX.MENSAL (AUT) (MB)  \
 2019-01-31                                1010.942608   
 2019-02-28                                1010.436607   
 2019-03-31                                1011.083737   
 2019-04-30                                1010.681667   
 2019-05-31                                1010.265524   
 ...                                               ...   
 2024-07-31                                1012.405645   
 2024-08-31                                1012.050672   
 2024-09-30                                1010.602917   
 2024-10-31                                1010.278091   
 2024-11-30                                1008.872917   
 
             TEMPERATURA ORVALHO MIN. MENSAL (AUT) (°C)  \
 2019-01-31                                   22.748387   
 2019-02-28                                   22.705208   
 2019-03-31                                   22.641801   
 2019-04-30                                   22.98888

# Criando modelo

In [ ]:
# Valor da seed para reprodutibilidade
seed_value = 42

# =================================================
# 1. Funções Auxiliares
# =================================================
def sanitizar_nomes_features(df):
    """Remove caracteres especiais dos nomes das colunas."""
    df.columns = [re.sub(r'[^a-zA-Z0-9]', '_', col).strip('_') for col in df.columns]
    return df

class SelectiveScaler(BaseEstimator, TransformerMixin):
    """Escalonador que ignora determinadas features (Mes_cos, Ano_diff, Mes_sin)."""
    def __init__(self):
        self.features_to_ignore = ['Mes_cos', 'Ano_diff', 'Mes_sin']
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.features_to_scale_ = [col for col in X.columns if col not in self.features_to_ignore]
        if self.features_to_scale_:
            self.scaler.fit(X[self.features_to_scale_])
        return self

    def transform(self, X):
        X_transformed = X.copy()
        if self.features_to_scale_:
            X_transformed[self.features_to_scale_] = self.scaler.transform(X[self.features_to_scale_])
        return X_transformed

# =================================================
# 2. Função Única para Processar DataFrame
# =================================================
def executar_modelo_estado(df_estado, target_col='FOCOS_DE_QUEIMADA', test_size=12):
    """
    Recebe um DataFrame e realiza:
      - Pré-processamento dos nomes das colunas;
      - Divisão temporal (treino e teste);
      - Otimização dos hiperparâmetros via Optuna;
      - Treinamento do modelo final com LightGBM;
      - Avaliação e cálculo das métricas;
      - Visualização interativa com eixo de datas alinhado e marcadores.

    Retorna um dicionário com:
      - 'modelo': pipeline final treinado,
      - 'features': features selecionadas,
      - 'metrics': métricas (RMSE, MAE, R²),
      - 'previsoes': valores previstos para o conjunto de teste,
      - 'real': valores reais do conjunto de teste,
      - 'datas': datas correspondentes ao teste.
    """
    # Pré-processamento
    df_estado = sanitizar_nomes_features(df_estado.copy())
    target_col = sanitizar_nomes_features(pd.DataFrame(columns=[target_col])).columns[0]

    # Split temporal (assumindo que o DataFrame esteja ordenado cronologicamente)
    train = df_estado.iloc[:-test_size]
    test = df_estado.iloc[-test_size:]

    X_train = train.drop(columns=[target_col])
    y_train = train[target_col]
    X_test = test.drop(columns=[target_col])
    y_test = test[target_col]

    # Otimização via Optuna
    def objective(trial):
        params = {
            'alpha': trial.suggest_float('alpha', 1e-5, 0.1, log=True),
            'l1_ratio': trial.suggest_float('l1_ratio', 0.1, 0.7),
            'num_leaves': trial.suggest_int('num_leaves', 2, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 15),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 0.1),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 0.1)
        }

        pipeline = Pipeline([
            ('scaler', SelectiveScaler()),
            ('selector', SelectFromModel(
                ElasticNet(alpha=params['alpha'], l1_ratio=params['l1_ratio'], max_iter=10000),
                threshold=0.001)),
            ('lgbm', LGBMRegressor(
                **{k: v for k, v in params.items() if k not in ['alpha','l1_ratio']},
                verbosity=-1,
                random_state=seed_value))
        ])

        try:
            scores = []
            cv = TimeSeriesSplit(n_splits=2)
            for train_idx, val_idx in cv.split(X_train):
                X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                pipeline.fit(X_tr, y_tr)
                preds = pipeline.predict(X_val)
                scores.append(np.sqrt(mean_squared_error(y_val, preds)))
            return np.mean(scores)
        except Exception as e:
            return np.inf

    study = optuna.create_study(direction='minimize',
                                sampler=optuna.samplers.TPESampler(seed=seed_value))
    study.optimize(objective, n_trials=15, show_progress_bar=False)

    # Seleção de features com ElasticNet
    best_en = ElasticNet(alpha=study.best_params['alpha'],
                         l1_ratio=study.best_params['l1_ratio'],
                         max_iter=10000)
    selector = SelectFromModel(best_en, threshold=0.001).fit(X_train, y_train)
    selected_features = X_train.columns[selector.get_support()].tolist()

    # Se nenhuma feature for selecionada, usa todas as features
    if len(selected_features) == 0:
        selected_features = X_train.columns.tolist()
        print("[AVISO] Nenhuma feature foi selecionada automaticamente. Usando todas as features:")
        print(list(selected_features))
    else:
        print("Features selecionadas:")
        print(list(selected_features))

    # Pipeline final com LightGBM
    final_model = Pipeline([
        ('scaler', SelectiveScaler()),
        ('lgbm', LGBMRegressor(
            **{k: v for k, v in study.best_params.items() if k not in ['alpha','l1_ratio']},
            verbosity=-1,
            random_state=seed_value))
    ])

    final_model.fit(X_train[selected_features], y_train)
    preds = final_model.predict(X_test[selected_features])

    # Cálculo das métricas
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    print("Métricas:")
    print(f"  RMSE : {rmse:.2f}")
    print(f"  MAE  : {mae:.2f}")
    print(f"  R²   : {r2:.2f}")

    # Visualização interativa: gráfico com eixo de datas alinhado e marcadores
    df_plot = pd.DataFrame({
        'Data': test.index,
        'Real': y_test.values,
        'Previsto': preds
    })

    # Converte a coluna 'Data' para datetime, se necessário
    if not np.issubdtype(df_plot['Data'].dtype, np.datetime64):
        df_plot['Data'] = pd.to_datetime(df_plot['Data'], errors='coerce')

    fig = px.line(
        df_plot,
        x='Data',
        y=['Real', 'Previsto'],
        title='Previsões vs Real',
        labels={'value': 'Focos de Queimada', 'variable': 'Tipo'},
        template='plotly_dark',
        markers=True
    )

    # Configuração do eixo x para datas no final do mês
    tick_vals = pd.date_range(start="2023-12-31", end="2024-11-30", freq='M')
    tick_text = [d.strftime('%b %Y') for d in tick_vals]
    fig.update_xaxes(
        tickmode='array',
        tickvals=tick_vals,
        ticktext=tick_text,
        range=["2023-12-31", "2024-11-30"]
    )

    # Atualiza o layout para aumentar a margem inferior e posicionar a anotação fora da área de plotagem
    fig.update_layout(
        margin=dict(l=80, r=80, t=100, b=200),  # margem inferior aumentada
        title=dict(
            text="Previsões vs Real",
            x=0.5,
            y=0.95,
            xanchor="center",
            yanchor="top"
        ),
        annotations=[
            dict(
                x=0.5,            # centralizada horizontalmente
                y=-0.2,           # posicionada fora da área dos eixos (abaixo)
                xref='paper',
                yref='paper',
                xanchor='center',
                yanchor='top',
                text=(f"<b>Métricas:</b><br>"
                      f"RMSE: {rmse:.2f}<br>"
                      f"MAE: {mae:.2f}<br>"
                      f"R²: {r2:.2f}"),
                showarrow=False,
                bgcolor='rgba(0,0,0,0.5)',
                bordercolor='white',
                borderwidth=1,
                borderpad=4
            )
        ]
    )

    fig.show()

    # Retorna os resultados para eventuais análises adicionais
    return {
        'modelo': final_model,
        'features': selected_features,
        'metrics': {'RMSE': rmse, 'MAE': mae, 'R2': r2},
        'previsoes': preds,
        'real': y_test.values,
        'datas': test.index
    }


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# AC Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['AC_final'])

[I 2025-02-07 21:24:50,853] A new study created in memory with name: no-name-58e1d123-1913-454d-953c-3ab57b483de7
[I 2025-02-07 21:24:50,967] Trial 0 finished with value: 874.6504922807942 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 874.6504922807942.
[I 2025-02-07 21:24:51,017] Trial 1 finished with value: 1270.5935811017575 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 874.6504922807942.
[I 2025-02-07 21:24:51,072] Trial 2 finished with value: 935.8433405496062 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, 'l

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 266.67
  MAE  : 188.44
  R²   : 0.95


# AL Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['AL_final'])

[I 2025-02-07 21:24:54,690] A new study created in memory with name: no-name-9093219b-2a57-4bba-9b1b-403857a7fe26
[I 2025-02-07 21:24:54,749] Trial 0 finished with value: 10.458510931349627 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 10.458510931349627.
[I 2025-02-07 21:24:54,806] Trial 1 finished with value: 9.474856522642053 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 1 with value: 9.474856522642053.
[I 2025-02-07 21:24:54,858] Trial 2 finished with value: 9.45211594557805 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, 'l

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 7.09
  MAE  : 5.24
  R²   : 0.82


# AM Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['AM_final'])

[I 2025-02-07 21:24:55,742] A new study created in memory with name: no-name-b3c74949-79bd-403a-af9c-03dc4513cb22
[I 2025-02-07 21:24:55,799] Trial 0 finished with value: 1510.5255143369177 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 1510.5255143369177.
[I 2025-02-07 21:24:55,856] Trial 1 finished with value: 2192.562571758862 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 1510.5255143369177.
[I 2025-02-07 21:24:55,906] Trial 2 finished with value: 1676.00050997052 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, '

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 878.10
  MAE  : 481.39
  R²   : 0.93


# AP Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['AP_final'])

[I 2025-02-07 21:24:56,781] A new study created in memory with name: no-name-f7219530-6803-4ee8-9f3c-3d86ea64cf3f
[I 2025-02-07 21:24:56,865] Trial 0 finished with value: 149.9787824552911 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 149.9787824552911.
[I 2025-02-07 21:24:56,909] Trial 1 finished with value: 219.46683481853947 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 149.9787824552911.
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number 

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 151.05
  MAE  : 71.02
  R²   : 0.81


# BA Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['BA_final'])

[I 2025-02-07 21:24:58,069] A new study created in memory with name: no-name-5701ddbf-0058-416e-a626-3a57c7cfb097
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.054e+05, tolerance: 7.109e+02

[I 2025-02-07 21:24:58,144] Trial 0 finished with value: 1139.3710113621416 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 1139.3710113621416.
[I 2025-02-07 21:24:58,189] Trial 1 finished with value: 1351.4525119941595 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_chil

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 597.56
  MAE  : 448.70
  R²   : 0.44


# CE Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['CE_final'])

[I 2025-02-07 21:24:59,267] A new study created in memory with name: no-name-cc9a21ea-9434-4eba-b687-9f777a760dd8
[I 2025-02-07 21:24:59,326] Trial 0 finished with value: 406.59086997923384 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 406.59086997923384.
[I 2025-02-07 21:24:59,369] Trial 1 finished with value: 524.9903200564834 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 406.59086997923384.
[I 2025-02-07 21:24:59,425] Trial 2 finished with value: 423.762107772697 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, '

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 510.00
  MAE  : 495.85
  R²   : 0.59


# DF Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['DF_final'])

[I 2025-02-07 21:25:00,355] A new study created in memory with name: no-name-0c97eb7c-6f77-43e1-b2c5-cd22d0539bde
[I 2025-02-07 21:25:00,420] Trial 0 finished with value: 19.924844915808304 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 19.924844915808304.
[I 2025-02-07 21:25:00,467] Trial 1 finished with value: 25.330528403505724 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 19.924844915808304.
[I 2025-02-07 21:25:00,528] Trial 2 finished with value: 19.95079867688626 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4,

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 19.99
  MAE  : 10.23
  R²   : 0.81


# ES Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['ES_final'])

[I 2025-02-07 21:25:01,745] A new study created in memory with name: no-name-3cce7f44-c916-410a-86c8-17493208d9a3
[I 2025-02-07 21:25:01,808] Trial 0 finished with value: 41.468779352433636 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 41.468779352433636.
[I 2025-02-07 21:25:01,852] Trial 1 finished with value: 34.954798223102216 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 1 with value: 34.954798223102216.
[I 2025-02-07 21:25:01,916] Trial 2 finished with value: 36.29761997820425 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4,

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 49.60
  MAE  : 30.38
  R²   : 0.41


# GO Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['GO_final'])

[I 2025-02-07 21:25:02,873] A new study created in memory with name: no-name-9f5bfb83-9cdc-42e0-ac09-28cba8eb0cce
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.325e+05, tolerance: 1.735e+03

[I 2025-02-07 21:25:02,942] Trial 0 finished with value: 486.80833945077035 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 486.80833945077035.
[I 2025-02-07 21:25:02,989] Trial 1 finished with value: 495.2546699578988 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 599.43
  MAE  : 477.66
  R²   : 0.47


# MA Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['MA_final'])

[I 2025-02-07 21:25:04,009] A new study created in memory with name: no-name-0f9c70c8-92d1-404c-b8c7-eb7d706018df
[I 2025-02-07 21:25:04,100] Trial 0 finished with value: 574.3210032384817 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 574.3210032384817.
[I 2025-02-07 21:25:04,170] Trial 1 finished with value: 1109.8084200535245 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 574.3210032384817.
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number 

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 402.67
  MAE  : 274.51
  R²   : 0.94


# MG Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['MG_final'])

[I 2025-02-07 21:25:05,827] A new study created in memory with name: no-name-dbbef56d-5a79-4328-b622-8b94cfeed14b
[I 2025-02-07 21:25:05,915] Trial 0 finished with value: 683.5744893729877 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 683.5744893729877.
[I 2025-02-07 21:25:05,984] Trial 1 finished with value: 879.0746428704999 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 683.5744893729877.
[I 2025-02-07 21:25:06,090] Trial 2 finished with value: 736.4819083373882 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, 'le

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 513.62
  MAE  : 295.63
  R²   : 0.87


# MS Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['MS_final'])

[I 2025-02-07 21:25:07,402] A new study created in memory with name: no-name-3cf4bb68-2230-462a-b6b9-4ed6ff3a7d2d
[I 2025-02-07 21:25:07,463] Trial 0 finished with value: 1060.6028953859595 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 1060.6028953859595.
[I 2025-02-07 21:25:07,507] Trial 1 finished with value: 927.0099673829095 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 1 with value: 927.0099673829095.
[I 2025-02-07 21:25:07,568] Trial 2 finished with value: 1040.721931792214 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, '

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 1505.85
  MAE  : 897.83
  R²   : -0.28


# MT Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['MT_final'])

[I 2025-02-07 21:25:08,537] A new study created in memory with name: no-name-d05062fd-b746-41ae-8273-08e67e1aaea6
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.181e+06, tolerance: 4.941e+04

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.279e+06, tolerance: 5.910e+04

[I 2025-02-07 21:25:08,620] Trial 0 finished with value: 2756.241046014283 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904,

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 4988.01
  MAE  : 2384.54
  R²   : 0.31


# PA Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['PA_final'])

[I 2025-02-07 21:25:09,740] A new study created in memory with name: no-name-02edea55-b209-4bf9-94a3-5619942c6718
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.019e+07, tolerance: 2.969e+04

[I 2025-02-07 21:25:09,812] Trial 0 finished with value: 2710.7563314056033 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 2710.7563314056033.
[I 2025-02-07 21:25:09,857] Trial 1 finished with value: 3052.4419384675402 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_chil

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 3921.96
  MAE  : 2283.39
  R²   : 0.54


# PB Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['PE_final'])

[I 2025-02-07 21:25:11,028] A new study created in memory with name: no-name-15811879-93da-4895-8799-74062e575237
[I 2025-02-07 21:25:11,091] Trial 0 finished with value: 111.81587438685204 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 111.81587438685204.
[I 2025-02-07 21:25:11,135] Trial 1 finished with value: 133.49518208546294 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 111.81587438685204.
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the numb

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 73.84
  MAE  : 47.36
  R²   : 0.87


# PE Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['PE_final'])

[I 2025-02-07 21:25:12,163] A new study created in memory with name: no-name-79fba72f-30b4-4b16-8198-9ee36b891d23
[I 2025-02-07 21:25:12,233] Trial 0 finished with value: 111.81587438685204 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 111.81587438685204.
[I 2025-02-07 21:25:12,280] Trial 1 finished with value: 133.49518208546294 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 111.81587438685204.
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the numb

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 73.84
  MAE  : 47.36
  R²   : 0.87


# PI Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['PI_final'])

[I 2025-02-07 21:25:13,320] A new study created in memory with name: no-name-f42083d2-b388-4d42-bbdb-46e684d749a7
[I 2025-02-07 21:25:13,379] Trial 0 finished with value: 682.4455834606534 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 682.4455834606534.
[I 2025-02-07 21:25:13,434] Trial 1 finished with value: 1188.5886122347688 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 682.4455834606534.
[I 2025-02-07 21:25:13,490] Trial 2 finished with value: 704.3237232585735 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, 'l

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 561.47
  MAE  : 287.88
  R²   : 0.66


# PR Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['PR_final'])

[I 2025-02-07 21:25:14,402] A new study created in memory with name: no-name-5ddb1f6f-ac0d-46b6-9f1e-4585ac2c23db
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.819e+04, tolerance: 1.882e+02

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.597e+03, tolerance: 3.546e+02

[I 2025-02-07 21:25:14,495] Trial 0 finished with value: 186.5990283742156 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904,

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 99.91
  MAE  : 69.85
  R²   : 0.87


# RJ Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['RJ_final'])

[I 2025-02-07 21:25:15,749] A new study created in memory with name: no-name-50de9eee-ea20-4f74-a2d5-9187e970fba1
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.086e+03, tolerance: 1.122e+01

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.888e+03, tolerance: 1.617e+01

[I 2025-02-07 21:25:15,830] Trial 0 finished with value: 39.53753807618091 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904,

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 80.60
  MAE  : 56.84
  R²   : 0.50


# RN Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['RN_final'])

[I 2025-02-07 21:25:17,025] A new study created in memory with name: no-name-95f88c23-2fc4-4b89-92e6-4aadc096366a
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.025e+03, tolerance: 1.006e+01

[I 2025-02-07 21:25:17,111] Trial 0 finished with value: 34.56425186267525 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 34.56425186267525.
[I 2025-02-07 21:25:17,177] Trial 1 finished with value: 46.73704725166239 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_s

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 31.68
  MAE  : 26.94
  R²   : 0.64


# RO Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['RO_final'])

[I 2025-02-07 21:25:18,858] A new study created in memory with name: no-name-6fe9a9f6-8453-445a-af1f-d19af5f7b03f
[I 2025-02-07 21:25:18,957] Trial 0 finished with value: 800.1226420321065 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 800.1226420321065.
[I 2025-02-07 21:25:19,028] Trial 1 finished with value: 1173.2539335248025 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 800.1226420321065.
[I 2025-02-07 21:25:19,139] Trial 2 finished with value: 913.908843474068 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, 'le

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 544.20
  MAE  : 335.30
  R²   : 0.84


# RR Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['RR_final'])

[I 2025-02-07 21:25:20,504] A new study created in memory with name: no-name-789f97ce-a962-49de-aa9a-fd5c4426fc0e
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.442e+03, tolerance: 6.551e+02

[I 2025-02-07 21:25:20,574] Trial 0 finished with value: 447.7416157627881 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 447.7416157627881.
[I 2025-02-07 21:25:20,619] Trial 1 finished with value: 179.58291181045908 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 571.68
  MAE  : 308.22
  R²   : 0.16


# RS Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['RS_final'])

[I 2025-02-07 21:25:21,629] A new study created in memory with name: no-name-6c0bfd8a-b790-41d2-99bb-8b39fbbc6662
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.202e+04, tolerance: 3.760e+02

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.619e+04, tolerance: 4.361e+02

[I 2025-02-07 21:25:21,722] Trial 0 finished with value: 220.55019739384016 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 75.71
  MAE  : 59.76
  R²   : 0.85


# SC Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['SC_final'])

[I 2025-02-07 21:25:22,953] A new study created in memory with name: no-name-34d89db7-c5a6-45bb-8a2b-07337c9c442d
[I 2025-02-07 21:25:23,011] Trial 0 finished with value: 134.7821267201531 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 134.7821267201531.
[I 2025-02-07 21:25:23,057] Trial 1 finished with value: 162.15259554420584 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 134.7821267201531.
[I 2025-02-07 21:25:23,111] Trial 2 finished with value: 165.24660510559966 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, '

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 64.03
  MAE  : 46.36
  R²   : 0.90


# SE Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['SE_final'])

[I 2025-02-07 21:25:24,023] A new study created in memory with name: no-name-0cae157c-2ba5-4563-949a-d6219a25ad19
[I 2025-02-07 21:25:24,076] Trial 0 finished with value: 11.958548135960882 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 11.958548135960882.
[I 2025-02-07 21:25:24,122] Trial 1 finished with value: 11.098254198383264 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 1 with value: 11.098254198383264.
[I 2025-02-07 21:25:24,175] Trial 2 finished with value: 11.62172897962363 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4,

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 10.63
  MAE  : 6.87
  R²   : 0.16


# SP Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['SP_final'])

[I 2025-02-07 21:25:25,040] A new study created in memory with name: no-name-b043e90a-7d2c-4457-84de-775eefbb4c8c
[I 2025-02-07 21:25:25,095] Trial 0 finished with value: 468.23576449201363 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 468.23576449201363.
[I 2025-02-07 21:25:25,139] Trial 1 finished with value: 475.4778711049272 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 468.23576449201363.
[I 2025-02-07 21:25:25,193] Trial 2 finished with value: 440.03491456631394 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4,

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 733.42
  MAE  : 406.54
  R²   : 0.55


# TO Resultados

In [ ]:
resultados = executar_modelo_estado(dataframes['TO_final'])

[I 2025-02-07 21:25:26,093] A new study created in memory with name: no-name-37cbec5b-b66c-4042-a653-2ed11c458020
[I 2025-02-07 21:25:26,163] Trial 0 finished with value: 748.9985834462102 and parameters: {'alpha': 0.00031489116479568613, 'l1_ratio': 0.6704285838459496, 'num_leaves': 7, 'learning_rate': 0.12374511199743694, 'min_child_samples': 6, 'reg_alpha': 0.015607892088416904, 'reg_lambda': 0.005817780380698264}. Best is trial 0 with value: 748.9985834462102.
[I 2025-02-07 21:25:26,208] Trial 1 finished with value: 890.4268997528493 and parameters: {'alpha': 0.029154431891537533, 'l1_ratio': 0.4606690070459253, 'num_leaves': 6, 'learning_rate': 0.013911053916202464, 'min_child_samples': 15, 'reg_alpha': 0.08324593965363418, 'reg_lambda': 0.021241787676720834}. Best is trial 0 with value: 748.9985834462102.
[I 2025-02-07 21:25:26,272] Trial 2 finished with value: 626.0800918265905 and parameters: {'alpha': 5.3370327626039544e-05, 'l1_ratio': 0.2100427059120603, 'num_leaves': 4, 'le

Features selecionadas:
['PRESS_O_ATMOSFERICA_MAX_MENSAL__AUT___MB', 'TEMPERATURA_ORVALHO_MIN__MENSAL__AUT____C', 'UMIDADE_REL__MIN__MENSAL__AUT', 'VENTO__VELOCIDADE_MENSAL__M_S', 'RADIACAO_GLOBAL__KJ_M', 'PRECIPITA__O_TOTAL__MENSAL__MM', 'TEMPERATURA_M_NIMA_MENSAL__AUT____C', 'Mes_sin', 'Mes_cos', 'Ano_diff']
Métricas:
  RMSE : 981.46
  MAE  : 532.97
  R²   : 0.71


# Comentários

Na primeira seleção manual de features, utilizamos a covariância entre as variáveis observadas no EDA anterior. Os resultados são mistos entre os estados por alguns motivos. No EDA, verificamos que alguns estados apresentavam grandes blocos de informações faltantes em determinadas colunas exógenas, os quais foram preenchidos por interpolação.

Além disso, enfrentamos o problema das previsões, pois precisamos alimentar o modelo com as colunas exógenas futuras. Isso nos obriga a depender de previsões externas ou a gerá-las por conta própria, situação que pode causar um acúmulo de erros e afetar a previsão final dos focos.

Embora um modelo univariado não enfrente esse problema, ele é mais limitado em suas previsões.